# Train test split KVLCC2 HSVA
<cite id="hzn48"><a href="#zotero|7505983/U2L55C9Q">(Luo et al., 2016)</a></cite>

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
ship="kvlcc2_hsva"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

train = "HSVA_CPMC_KVLCC2_Z_25_05"
df_train = catalog.load(f'{ship}.updated.{train}.data_ek_smooth')

test = "HSVA_CPMC_KVLCC2_Z_35_05"
df_test = catalog.load(f'{ship}.updated.{test}.data_ek_smooth')

#delta_start = np.deg2rad(10.5)
#
#for data in [df_train, df_test]:
#    index_start = (data['delta'].diff().abs() > 0).idxmax()
#    start = data.loc[0:index_start] 
#    start['delta'] = delta_start

#index_start = (df_train['delta'].diff().abs() > 0).idxmax()
#df_train = df_train.loc[index_start:].copy()
#df_train.index = df_train.index - df_train.index[0]

index_start = (df_test['delta'].diff().abs() > 0).idxmax()
df_test = df_test.loc[index_start:].copy()
df_test.index = df_test.index - df_test.index[0]


added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")


In [ ]:
data = df_train
data_with_force = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)


In [ ]:
regression, parameters = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
model = create_model_from_motion_regression(regression=regression)

In [ ]:
df_ = df_test.copy()
df_.iloc[0]['r'] = 0
result = model.simulate(df_=df_)
df_result = result.result.copy()
df_result['psi_deg'] = np.rad2deg(df_result['psi'])
df_test['psi_deg'] = np.rad2deg(df_test['psi'])
df_result['delta_deg'] = np.rad2deg(df_result['delta'])
df_test['delta_deg'] = np.rad2deg(df_test['delta'])


In [ ]:
dataframes = {
    'Experiment' : df_test,
    'Prediction' : df_result
}

styles = {
    'Experiment' : {'style':'r--'},
    'Prediction' : {'style':'b-'},
}

In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles, time_window=[0,2]);
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
parameters